In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
tickers_1 = ['GOOGL', 'AAPL', 'AMZN', 'NVDA', 'BTC-USD', '^IXIC', 'AKBNK.IS', 'TUPRS.IS', '^SPX']
ticker2 = ['XU100.IS']
us_stocks = ['GOOGL', 'AAPL', 'AMZN', 'NVDA']
tr_stocks = ['AKBNK.IS', 'TUPRS.IS']

In [ ]:
data = yf.download(tickers_1, start='2017-01-01')
data = data.loc[:, 'Adj Close']

xu100 = yf.download(ticker2, start='2021-01-01')['Adj Close'].to_frame(name = 'XU100.IS')
xu100 = xu100.asfreq('D')
xu100.index = pd.to_datetime(xu100.index).tz_localize('UTC')

dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
data = data.reindex(dates)
xu100 = xu100.reindex(dates)

data = pd.concat([data, xu100], axis=1)


In [4]:
daily_data = data
weeekly_data = data.resample('W').last()
monthly_data = data.resample('ME').last()
yearly_data = data.resample('YE').last()

In [ ]:
daily_returns = daily_data.pct_change()
weekly_returns = weeekly_data.pct_change()
monthly_returns = monthly_data.pct_change()


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(9,6)) 
ax[0].plot(daily_data['^SPX'], label='SP500', color='blue')
ax[0].set_title('US Index: SP500')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Price')
ax[0].legend()

# Plot US Stocks
ax[1].plot(daily_data['GOOGL'], label='Google', color='green')
ax[1].plot(daily_data['AAPL'], label='Apple', color='orange')
ax[1].plot(daily_data['AMZN'], label='Amazon', color='red')
ax[1].plot(daily_data['NVDA'], label='Nvidia', color='yellow')
ax[1].set_title('US Stocks')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Price')
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15,10)) 
ax[0].plot(daily_returns['^SPX'], label='SP500', linewidth=0.5, color='blue')
ax[0].set_title('US Index: SP500')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('% Returns')
ax[0].legend()

# Plot US Stocks
ax[1].plot(daily_returns['GOOGL'], label='Google', linewidth=0.5, color='green')
ax[1].plot(daily_returns['AAPL'], label='Apple', linewidth=0.5, color='orange')
ax[1].plot(daily_returns['AMZN'], label='Amazon', linewidth=0.5, color='red')
ax[1].plot(daily_returns['NVDA'], label='Nvidia', linewidth=0.5, color='yellow')
ax[1].set_title('US Stocks')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('% Returns')
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(9,6)) 
ax[0].plot(daily_data['XU100.IS'], label='BIST', color='blue')
ax[0].set_title('TR Index: BIST100')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Price')
ax[0].legend()

ax[1].plot(daily_data['AKBNK.IS'], label='AKBANK', color='green')
ax[1].plot(daily_data['TUPRS.IS'], label='TUPRAS', color='orange')
ax[1].set_title('TR Stocks')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Price')
ax[1].legend()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(9,6)) 
ax[0].plot(daily_returns['XU100.IS'], label='BIST', linewidth=0.5, color='blue')
ax[0].set_title('TR Index: BIST100')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Price')
ax[0].legend()

ax[1].plot(daily_returns[daily_returns.index>'2021-01-01']['AKBNK.IS'], label='AKBANK', linewidth=0.5, color='green')
ax[1].plot(daily_returns[daily_returns.index>'2021-01-01']['TUPRS.IS'], label='TUPRAS', linewidth=0.5, color='orange')
ax[1].set_title('TR Stocks')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Price')
ax[1].legend()

In [ ]:
print('prices')
print(daily_data[['AAPL', 'AMZN', 'BTC-USD', 'GOOGL', 'NVDA']].agg(['mean', 'std', 'var', 'skew']))
print(daily_data[['AKBNK.IS', 'TUPRS.IS']].agg(['mean', 'std', 'var', 'skew']))
print()
print('returns')
print(daily_returns[['AAPL', 'AMZN', 'BTC-USD', 'GOOGL', 'NVDA']].agg(['mean', 'std', 'var', 'skew']))
print(daily_returns[['AKBNK.IS', 'TUPRS.IS']].agg(['mean', 'std', 'var', 'skew']))

In [ ]:
# PART 2 QUESTION 5
y_apple = monthly_returns['AAPL']
y_amazon = monthly_returns['AMZN']
y_google = monthly_returns['GOOGL']
y_nvidia = monthly_returns['NVDA']
y_us = [y_apple, y_amazon, y_google, y_nvidia]

y_tupras = monthly_returns['TUPRS.IS'][monthly_returns.index>'2021-01-01'] # for turkish stocks
y_akbank = monthly_returns['AKBNK.IS'][monthly_returns.index>'2021-01-01'] # for turkish stocks
y_tr = [y_tupras, y_akbank]


x_bist = monthly_returns['XU100.IS'][monthly_returns.index>'2021-01-01'] # for turkish stocks
x_bist = sm.add_constant(x_bist)

x_nasdaq = monthly_returns['^IXIC']
x_nasdaq = sm.add_constant(x_nasdaq)

for y in y_us:    
    model = sm.OLS(y.dropna(), x_nasdaq.dropna())
    results = model.fit()
    print(y.name, results.params['^IXIC'])

for y in y_tr:
    model = sm.OLS(y.iloc[1:], x_bist.dropna())
    results = model.fit()
    print(y.name, results.params['XU100.IS'])
    